In [82]:
import sqlite3
from pymystem3 import Mystem
m = Mystem()
con = sqlite3.connect('FDB.db')
cur = con.cursor()

In [2]:
import pandas as pd
import re

# 0 Data preparation

Tags

In [3]:
with open('./raw_data/tags.csv','r') as f:
    tags = f.readlines()
    result = {t.split()[0]:t.split()[1] for t in tags}
print (result)

{'A': 'pos', 'ADV': 'pos', 'ADVPRO': 'pos', 'ANUM': 'pos', 'APRO': 'pos', 'COM': 'pos', 'CONJ': 'pos', 'INTJ': 'pos', 'NUM': 'pos', 'PART': 'pos', 'PR': 'pos', 'S': 'pos', 'SPRO': 'pos', 'V': 'pos', 'praes': 'tense', 'inpraes': 'tense', 'praet': 'tense', 'nom': 'case', 'gen': 'case', 'dat': 'case', 'acc': 'case', 'ins': 'case', 'abl': 'case', 'part': 'case', 'loc': 'case', 'voc': 'case', 'sg': 'num', 'pl': 'num', 'ger': 'vform', 'inf': 'vform', 'partcp': 'vform', 'indic': 'vform', 'imper': 'vform', 'brev': 'aform', 'plen': 'aform', 'poss': 'aform', 'supr': 'degree', 'comp': 'degree', '1p': 'pers', '2p': 'pers', '3p': 'pers', 'm': 'gender', 'f': 'gender', 'n': 'gender', 'ipf': 'asp', 'pf': 'asp', 'act': 'voice', 'pass': 'voice', 'tran': 'trans', 'intr': 'trans', 'parenth': 'type', 'praed': 'type', 'abbr': 'type', 'mf': 'type', 'awkw': 'form', 'dist': 'form', 'geo': 'sem', 'persn': 'sem', 'patrn': 'sem', 'famn': 'sem', 'obsc': 'style', 'inform': 'style', 'rare': 'style', 'obsol': 'style'

# 1 block

**Import informants**

In [123]:
db_1 = sqlite3.connect('./raw_data/ABM.sqlite')
db_1_cur = db_1.cursor()

In [124]:
db_1_cur.execute('SELECT * FROM tblInformators')
inf = db_1_cur.fetchall()

In [127]:
cur.executemany('INSERT INTO Informators (code, name, city, bio, old_id) VALUES (?,?,?,?,?)', inf)

In [128]:
con.commit()

**Import texts to database**

load data

In [102]:
db_1_cur.execute('SELECT * FROM tblCards')
texts = db_1_cur.fetchall()

In [29]:
for key, value in enumerate(texts[0]):
    print (key, value)

0 [Шёл разговор о пьющей женщине из Карпиловки, Г назвал её «Ма\ндыка».] [Ма\ндыка?] [Г:] Да. [Что это значит?] [Г, смеясь:] Ну, манды\ка — э\то… [А одновременно:] Коро\че её ёб… [Г:] Все. [А:] Всё село\. [Г:] Да, коро\че… [А:] И… и… и из на\шего, и коро\че там… [Г:] Все подря\д. [А:] Да. [А и Г одновременно:] Кому\ не лень. [А:] Буты\лку на стол, и… [Она пьёт, да?] [Г:] Ну, она\ по-чё\рному пьёт.
1 
2 
3 
4 XXIa
5 5 б
6 Брянская обл., Злынковский р-н, Спиридонова Буда
7 2017
8 А
9 Г
10 
11 
12 МЭГ
13 НВП
14 СЕ
15 ФЕЕ
16 1
17 
18 
19 
20 


In [116]:
texts2 = []

In [117]:
for row in texts:
    new_row = (
        'АБМ',
        row[16],
        row[6],
        row[0],
        row[7],
        restore_label(row[5]),
        row[4],
        ', '.join(i for i in row[8:12] if i != ''),
        ', '.join(i for i in row[12:16] if i != '') 
    )
    texts2.append(new_row)

In [121]:
cur.executemany('INSERT INTO Texts (leader, old_id, city, raw_text, year, question, list, informants, collectors)\
VALUES (?,?,?,?,?,?,?,?,?)', texts2)

In [122]:
con.commit()

2. change question labels

In [110]:
def restore_label(label):
    label = label.split(',')
    result = []
    #print(label)
    for i in label:
        q = re.findall('^([0-9]*)[^0-9a-zа-я]*?([ a-zа-я\.]*?)$', i.strip())[0]
        #print (q)
        result.append((q[0].strip(), q[1].strip()))
    for key, value in enumerate(result):
        if value[0] == '':
            result[key] = (result[key-1][0].strip(), value[1].strip())
    #print (result)
    return ', '.join(''.join(i) for i in result)

In [115]:
restore_label('15 а, 19 а, в, 30 а, 14 а, 25 а, б, 16 а')

'15а, 19а, 19в, 30а, 14а, 25а, 25б, 16а'

In [68]:
re.findall('^([0-9]*)[^0-9a-zа-я]*?([a-zа-я]*?)$',label[1].strip())

[('10', 'б')]

In [ ]:
data['Вопрос'] = restore_label(data['Вопрос'])

3. normalize text

In [215]:
a = 'у% Я́ цветоу%ок не брал и цветоу%'
len ([i['text'] for i in m.analyze(a)]) == len(m.analyze(a))

True

In [216]:
def norm_v(text):
    k = [i['text'] for i in m.analyze(text)]
    r = ['empty']
    for i in k:
        if '%' not in i:
            if r[-1][-1] != '%':
                r += [i]
            else:
                r[-1]+=i
        else:
            r[-1]+='%'
            if i != '%':
                r += [i.replace('%','')] 
    return r[1:]

In [218]:
norm_v(a)

['у%',
 ' ',
 'Я́',
 ' ',
 'цветоу%ок',
 ' ',
 'не',
 ' ',
 'брал',
 ' ',
 'и',
 ' ',
 'цветоу%',
 '\n']

In [85]:
''.join(norm_v(a)).replace('у%','в')

'в Я́ цветовок не брал и цветов\n'

In [ ]:
def 

In [12]:
accents = {'А́ ':'А', 'Е́ ':'Е','И́ ':'И','О́ ':'О','У́ ':'У','Ы́ ':'Ы','Э́ ':'Э','Ю́ ':'Ю','Я́':'Я',
           'а́':'а','е́':'е','и́':'и','о́':'о', 'у́':'у', 'ы́':'ы', 'э́':'э', 'ю́':'ю', 'я́':'я'}

In [32]:
get_accents = {item[1]+'\\': item[0] for item in accents.items()}

In [168]:
a = '[Вы вчера рассказывали историю с цветами, что это было?] [А:] Ну, э\то как бы э… не зна\ю, и\ли приме\та и\ли… ну, что-то… вот когда\ моего\ дя\дю хорони\ли э… то\же, ну, кода\ хорони\ли его\, хоте\ли то\же цветы\ положи\ть и\ли что-то хоте\ли положи\ть туда\, вро\де, цветы\, она\ стро\го запрети\ла, ну там, же\нщина, кото\рая всё по э\тим, по похорона\м от, она\ сказа\ла: «Стро\го нельзя\». Ну, ти\па, ско\ка положи\ли тогда\, ну, в про\шлый раз там что-то, в моги\лу цвето\у%, сто… че\рез сто\ка дней ро\дственник у\мер и\ли чт… ну, коро\че, что-то тако\е, и\ли когда\ цветы\ завя\ли, ро\дственник у\мер, что-то… [Соб.: Ты что-то рассказывал, что она букет кинула в гроб…] Ну да, э\то вон, Гри\ша, да. [Соб.: А, Гриша.] [Г:] Я расска\зывал от что, ну, они\ сказа\ли то, что кода\ она\ ски\нула, ско\ка, то\ ли де\вять цвето\в, ну, в моги\лу получа\ется кода\... все песо\к кида\ли, а она\ цветы\ ски\нула. И по\сле э\того у нас, ну, начало\сь, вот э\ти де\вять пацано\в у\мерло. И\менно вот из-за э\того. Говоря\т то, что и\менно из-за э\того, то что…'

In [88]:
s = set(re.findall('.\\\\', a))
s

{'И\\', 'а\\', 'е\\', 'и\\', 'о\\', 'у\\', 'ы\\', 'э\\', 'я\\'}

In [108]:
for i in s:
    a = a.replace(i, get_accents[i])

In [109]:
a

'[Вы вчера рассказывали историю с цветами, что это было?] [А:] Ну, э́то как бы э… не зна́ю, и́ли приме́та и́ли… ну, что-то… вот когда́ моего́ дя́дю хорони́ли э… то́же, ну, кода́ хорони́ли его́, хоте́ли то́же цветы́ положи́ть и́ли что-то хоте́ли положи́ть туда́, вро́де, цветы́, она́ стро́го запрети́ла, ну там, же́нщина, кото́рая всё по э́тим, по похорона́м от, она́ сказа́ла: «Стро́го нельзя́». Ну, ти́па, ско́ка положи́ли тогда́, ну, в про́шлый раз там что-то, в моги́лу цвето́у%, сто… че́рез сто́ка дней ро́дственник у́мер и́ли чт… ну, коро́че, что-то тако́е, и́ли когда́ цветы́ завя́ли, ро́дственник у́мер, что-то… [Соб.: Ты что-то рассказывал, что она букет кинула в гроб…] Ну да, э́то вон, Гри́ша, да. [Соб.: А, Гриша.] [Г:] Я расска́зывал от что, ну, они́ сказа́ли то, что кода́ она́ ски́нула, ско́ка, то́ ли де́вять цвето́в, ну, в моги́лу получа́ется кода́... все песо́к кида́ли, а она́ цветы́ ски́нула. И по́сле э́того у нас, ну, начало́сь, вот э́ти де́вять пацано́в у́мерло. И́ менно вот из

In [113]:
a = re.sub('\[','\n[',a)

a = re.sub('[^:]\]',']\n',a)

a = re.sub('[ ]*?\n','\n',a)

a = re.sub('\n[ \n]*','\n',a).strip()

a = a.split('\n')

In [151]:
for i in a:
    r = re.findall('\[.*?\]', i)
    if len(r) > 0:
        if re.match('^\[[A-ZА-Я-0-9]{1,5}:\]$', r[0]):
            print (r[0])

[А:]
[Г:]


In [41]:
m.analyze(a)[22:25]

[{'analysis': [{'lex': 'ну', 'wt': 0.9991076439, 'gr': 'PART='}],
  'text': 'Ну'},
 {'text': ', '},
 {'analysis': [{'lex': 'это',
    'wt': 0.7809833731,
    'gr': 'SPRO,sg,n,inan=(acc|nom)'}],
  'text': 'э́то'}]

In [4]:
class OneSentence:
    def __init__(self, text, prefix):
        self.show_prefix = False
        self.prefix = prefix
        self.text = text
        
    def __str__(self):
        return '({}\t{}\t{})'.format(str(self.show_prefix), self.prefix, self.text)
    
    __repr__ = __str__
        
class Sentences:
    def __init__(self,text):
        self.sentences = self._assign_turns(text)
        #print (self.turns)
        
    def _split_turns(self, text):
        global get_accents
        tags = set(re.findall('.\\\\', text))
        for i in tags:
            text = text.replace(i, get_accents[i])
        text = re.sub('\[','\n[',text)
        text = re.sub('[^:]\]',']\n',text)
        text = re.sub('[ ]*?\n','\n',text)
        text = re.sub('\n[ \n]*','\n',text).strip()
        text = text.split('\n')
        return text
    
    def _assign_turns(self,text):
        text = self._split_turns(text)
        prefixes = []
        result = []
        for i in text:
            r = re.findall('\[.*?\]', i)
            if len(r) > 0:
                if re.match('^\[[A-ZА-Я-0-9]{1,5}:\]$', r[0]):
                    prefixes.append(r[0])
                    sentences = sent_tokenize(i.replace(r[0], '').strip())
                    result.append(OneSentence(sentences[0], prefixes[-1]))
                    result[-1].show_prefix = True
                    for x in sentences[1:]:
                        result.append(OneSentence(x, prefixes[-1]))
                else:
                    result.append(OneSentence(r[0], ''))
            else:
                sentences = sent_tokenize(i.strip())
                result.append(OneSentence(sentences[0], prefixes[-1]))
                result[-1].show_prefix = True
                for x in sentences[1:]:
                    result.append(OneSentence(x, prefixes[-1]))
        return result
    
    def __str__(self):
        return '\n\n'.join([str(i) for i in self.sentences])
    
    __repr__ = __str__

In [5]:
class TaggedSentences:
    
    def __init__(self, sentences, informators, city):
        self.sentences = sentences
        self.meta = self._get_inform(informators, city)
        
    def _norm_v(self, text):
        global accents
        k = [i['text'] for i in m.analyze(text)]
        r = ['empty']
        for i in k:
            if '%' not in i:
                if r[-1][-1] != '%':
                    r += [i]
                else:
                    r[-1]+=i
            else:
                r[-1]+='%'
                if i != '%':
                    r += [i.replace('%','')] 
        norm = ''.join(r[1:]).replace('у%','в').replace('У%','В')
        for i in accents:
            norm = norm.replace(i, accents[i])
        return r[1:], m.analyze(norm)
    
    def _get_inform(self, informators, city):
        global cur
        informators = [i.strip() for i in informators.split(',')]
        meta = {}
        for i in informators:
            

SyntaxError: invalid syntax (<ipython-input-5-f19f4878dc41>, line 25)

In [10]:
def _get_inform(informators, city):
        global cur
        informators = [i.strip() for i in informators.split(',')]
        meta = {}
        city = city.split(',')[-1].strip()
        for i in informators:
            cur.execute('SELECT id, code, birthplace, birthyear, gender FROM Informators WHERE\
            code=\"{}\" AND city=\"{}\"'.format(i, city))
            print (cur.fetchall())

In [15]:
cur.execute('SELECT informants, city from Texts')
data = cur.fetchall()

In [21]:
for i in data:
    _get_inform(*i)

[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(154, 'ГМА', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(154, 'ГМА', None, None, None)]
[(154, 'ГМА', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(154, 'ГМА', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(1, 'Г', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)]
[(2, 'А', None, None, None)]
[(1, 'Г', None, None, None)]
[(2, 'А', None, None, None)

[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(218, 'ЛП', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(218, 'ЛП', None, None, None)]
[(202, 'АВ', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, None)]
[(202, 'АВ', None, None, None)]
[(198, 'КАК', None, None, None)]
[(202, 'АВ', None, None, No

[(14, 'ЗН', None, None, None)]
[(14, 'ЗН', None, None, None)]
[(14, 'ЗН', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', None, None, None)]
[(142, 'ГВИ', No

[(225, 'РНС', None, None, None)]
[(225, 'РНС', None, None, None)]
[(225, 'РНС', None, None, None)]
[(225, 'РНС', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93, 'ДЛА', None, None, None)]
[(93

[(87, 'В', None, None, None)]
[(84, 'КТМ', None, None, None)]
[(85, 'КВИ', None, None, None)]
[(86, 'РВА', None, None, None)]
[(87, 'В', None, None, None)]
[(84, 'КТМ', None, None, None)]
[(85, 'КВИ', None, None, None)]
[(86, 'РВА', None, None, None)]
[(87, 'В', None, None, None)]
[(84, 'КТМ', None, None, None)]
[(85, 'КВИ', None, None, None)]
[(86, 'РВА', None, None, None)]
[(87, 'В', None, None, None)]
[(85, 'КВИ', None, None, None)]
[(87, 'В', None, None, None)]
[(84, 'КТМ', None, None, None)]
[(85, 'КВИ', None, None, None)]
[(86, 'РВА', None, None, None)]
[(87, 'В', None, None, None)]
[(85, 'КВИ', None, None, None)]
[(86, 'РВА', None, None, None)]
[(84, 'КТМ', None, None, None)]
[(85, 'КВИ', None, None, None)]
[(84, 'КТМ', None, None, None)]
[(114, 'МВ', None, None, None)]
[(85, 'КВИ', None, None, None)]
[(84, 'КТМ', None, None, None)]
[(87, 'В', None, None, None)]
[(84, 'КТМ', None, None, None)]
[(85, 'КВИ', None, None, None)]
[(87, 'В', None, None, None)]
[(84, 'КТМ', None, None,

[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(206, 'ДМД', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None, None)]
[(75, 'ДНВ', None, None

[(190, 'ЦПИ', None, None, None)]
[(189, 'МТ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(189, 'МТ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(189, 'МТ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(189, 'МТ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(190, 'ЦПИ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', None, None, None)]
[(243, 'МТЯ', 

[(182, 'С', None, None, None)]
[(213, 'А', None, None, None)]
[(211, 'И', None, None, None)]
[(211, 'И', None, None, None)]
[(182, 'С', None, None, None)]
[(213, 'А', None, None, None)]
[(181, 'Д', None, None, None)]
[(213, 'А', None, None, None)]
[(182, 'С', None, None, None)]
[(213, 'А', None, None, None)]
[(182, 'С', None, None, None)]
[(181, 'Д', None, None, None)]
[(181, 'Д', None, None, None)]
[(182, 'С', None, None, None)]
[(212, 'РД', None, None, None)]
[(211, 'И', None, None, None)]
[(213, 'А', None, None, None)]
[(182, 'С', None, None, None)]
[(181, 'Д', None, None, None)]
[(182, 'С', None, None, None)]
[(213, 'А', None, None, None)]
[(211, 'И', None, None, None)]
[(181, 'Д', None, None, None)]
[(213, 'А', None, None, None)]
[(182, 'С', None, None, None)]
[(212, 'РД', None, None, None)]
[(211, 'И', None, None, None)]
[(213, 'А', None, None, None)]
[(182, 'С', None, None, None)]
[(182, 'С', None, None, None)]
[(213, 'А', None, None, None)]
[(211, 'И', None, None, None)]
[(181,

In [227]:
def norm_v(text):
        global accents
        k = [i['text'] for i in m.analyze(text)]
        r = ['empty']
        for i in k:
            if '%' not in i:
                if r[-1][-1] != '%':
                    r += [i]
                else:
                    r[-1]+=i
            else:
                r[-1]+='%'
                if i != '%':
                    r += [i.replace('%','')] 
        norm = ''.join(r[1:]).replace('у%','в').replace('У%','В')
        for i in accents:
            norm = norm.replace(i, accents[i])
        return r[1:], m.analyze(norm)

In [229]:
a = 'У% Я́ цветоу%ок не брал и цвето́у%'
norm_v(a)

(['У%',
  ' ',
  'Я́',
  ' ',
  'цветоу%ок',
  ' ',
  'не',
  ' ',
  'брал',
  ' ',
  'и',
  ' ',
  'цвето́у%',
  '\n'],
 [{'analysis': [{'lex': 'в',
     'wt': 8.212235587e-06,
     'gr': 'S,abbr=(abl,pl|abl,sg|acc,pl|acc,sg|dat,pl|dat,sg|gen,pl|gen,sg|ins,pl|ins,sg|nom,pl|nom,sg)'}],
   'text': 'В'},
  {'text': ' '},
  {'analysis': [{'lex': 'я',
     'wt': 2.837188476e-05,
     'gr': 'S,abbr=(abl,pl|abl,sg|acc,pl|acc,sg|dat,pl|dat,sg|gen,pl|gen,sg|ins,pl|ins,sg|nom,pl|nom,sg)'}],
   'text': 'Я'},
  {'text': ' '},
  {'analysis': [{'lex': 'цветовка',
     'wt': 1,
     'qual': 'bastard',
     'gr': 'S,geo,f,inan=gen,pl'}],
   'text': 'цветовок'},
  {'text': ' '},
  {'analysis': [{'lex': 'не', 'wt': 1, 'gr': 'PART='}], 'text': 'не'},
  {'text': ' '},
  {'analysis': [{'lex': 'брать',
     'wt': 1,
     'gr': 'V,ipf,tran=praet,sg,indic,m'}],
   'text': 'брал'},
  {'text': ' '},
  {'analysis': [{'lex': 'и', 'wt': 0.9999770357, 'gr': 'CONJ='}], 'text': 'и'},
  {'text': ' '},
  {'analysis': 

In [181]:
from nltk.tokenize import sent_tokenize

In [210]:
A = Sentences(a)
A

(False		[Вы вчера рассказывали историю с цветами, что это было])

(True	[А:]	Ну, э́то как бы э… не зна́ю, и́ли приме́та и́ли… ну, что-то… вот когда́ моего́ дя́дю хорони́ли э… то́же, ну, кода́ хорони́ли его́, хоте́ли то́же цветы́ положи́ть и́ли что-то хоте́ли положи́ть туда́, вро́де, цветы́, она́ стро́го запрети́ла, ну там, же́нщина, кото́рая всё по э́тим, по похорона́м от, она́ сказа́ла: «Стро́го нельзя́».)

(False	[А:]	Ну, ти́па, ско́ка положи́ли тогда́, ну, в про́шлый раз там что-то, в моги́лу цвето́у%, сто… че́рез сто́ка дней ро́дственник у́мер и́ли чт… ну, коро́че, что-то тако́е, и́ли когда́ цветы́ завя́ли, ро́дственник у́мер, что-то…)

(False		[Соб.: Ты что-то рассказывал, что она букет кинула в гроб])

(True	[А:]	Ну да, э́то вон, Гри́ша, да.)

(False		[Соб.: А, Гриша])

(True	[Г:]	Я расска́зывал от что, ну, они́ сказа́ли то, что кода́ она́ ски́нула, ско́ка, то́ ли де́вять цвето́в, ну, в моги́лу получа́ется кода́... все песо́к кида́ли, а она́ цветы́ ски́нула.)

(False	[Г:]	И по́сл

In [213]:
text = A.sentences[2].text

In [ ]:
m.analyze

In [133]:
def no_stress(text):
    text = text.replace('\\', '')
    return text

def split_sentences(text):
    text = text.replace('[', '\n[')
    text = re.sub('\n{2,}','\n')
    text = text.split('\n')
    text = [sent_tokenize(chunk) for chunk in text]
    return text

In [ ]:
def get_meta(code):
    cur.execute('SELECT id, gender, birthyear FROM Informants WHERE code=\"{}\"'.format(code))
    indiv = cur.fetchone()
    return {'id':indiv[0], 'gender':indiv[1], 'year':indiv[2]}

def get_col_id(code):
    cur.execute('SELECT id FROM Collectors WHERE code=\"{}\"'.format(code))
    indiv = cur.fetchone()
    return indiv[0]

def joint_meta(informants, collectors):
    informants = informants.split(',')
    result = {'inf':{}, 'col':{}}
    for i in informants:
        result['inf'][i.strip()] = get_meta(code)
    for i in collectors:
        result['col'][i.strip()] = get_col_id(code)

In [ ]:
def join_text(text, sentence):
    text = text
    for word in words:
            if 'wf' not in word:
                continue
            word['off_start'] = len(text)
            if word['wtype'] == 'word':
                text += word['wf'] + ' '
                word['off_end'] = len(text) - 1
            else:
                if word['wf'].startswith(('(', '[', '{', '<', '“')):
                    text += word['wf']
                    word['off_end'] = len(text)
                elif word['wf'].startswith((')', ']', '}', '>', '.', ',', '?', '!', '”', '…')):
                    if text.endswith(' '):
                        word['off_start'] -= 1
                        text = text[:-1]
                    text += word['wf'] + ' '
                    word['off_end'] = len(text) - 1
                else:
                    text += word['wf'] + ' '
                    word['off_end'] = len(text) - 1
        return text.strip()

In [ ]:
def ana(analysis):
    global tags
    result = {}
    for i in analysis:
        if i in tags:
            result[tags[i]] = i
    return result

In [ ]:
def tokens(parsed):
    result = []
    for i in parsed:
        if 'analysis' in i:
            r = {}
            r['wf'] = i['text']
            r['wtype'] = i['word']
            r['ana'] = []
            for var in i['analysis']:
                r['ana'].append(ana(var))
    return result

In [ ]:
def parse_sentence(sentence, meta):
    result = {'meta':{}, 'words':[]}
    if not re.match('^\[.*?\]$', sentence):
        sentence = sentence.split(']')
        sentence[0] = sentence[0]+']'
        parsed = tokens(m.analyze(sentence[1]))
    else:
        parsed = tokens(m.analyze(sentence))
    

# Block 2

Informators

In [42]:
df = pd.read_csv('./raw_data/Список файлов - Лист1.csv') 

In [43]:
df.columns

Index(['Имя файла', 'Жанр', 'Ссылка на текстовый файл', 'Ссылка на аудио файл',
       'Деревня', 'ФИО', 'Год рождения', 'Место рождения',
       'Место записи (адрес, координаты, etc.)', 'ФИ собирателей',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')

In [44]:
df.head(3)

,Имя файла,Жанр,Ссылка на текстовый файл,Ссылка на аудио файл,Деревня,ФИО,Год рождения,Место рождения,"Место записи (адрес, координаты, etc.)",ФИ собирателей,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Архив ВШЭ-ФЭ-69-14-2017-1,Сведения об игре на гармони,https://github.com/expokimry17/texts/blob/mast...,NaN,пос.Белый городок,Кочнев Виктор Алексеевич,1934,Калязинский район,"пос. Белый городок, ул. Заводская д. 5","Надежда Григорьева, Аня Копаева",NaN,NaN,NaN
1,Архив ВШЭ-ФЭ-69-14-2017-2,Молодежные посиделки,https://github.com/expokimry17/texts/blob/mast...,NaN,пос.Белый городок,Кочнев Виктор Алексеевич,1934,Калязинский район,"пос. Белый городок, ул. Заводская д. 5","Надежда Григорьева, Аня Копаева",NaN,NaN,NaN
2,Архив ВШЭ-ФЭ-69-14-2017-3,"Сведения о песнях, танцах, репертуаре на молод...",https://github.com/expokimry17/texts/blob/mast...,NaN,пос.Белый городок,Кочнев Виктор Алексеевич,1934,Калязинский район,"пос. Белый городок, ул. Заводская д. 5","Надежда Григорьева, Аня Копаева",NaN,NaN,NaN


In [45]:
inf = df[['ФИО', 'Деревня','Год рождения', 'Место рождения']].drop_duplicates(['ФИО'])

In [ ]:
inf

In [51]:
for i in range(len(inf)):
    d = inf.iloc[i]
    if ',' not in d['ФИО']:
        s = 'INSERT INTO Informators (name, city, birthplace, birthyear) \
        VALUES (\"{}\",\"{}\",\"{}\",\"{}\")'.format(
        d['ФИО'], d['Деревня'], d['Место рождения'], d['Год рождения'])
        cur.execute(s)

In [52]:
con.commit()

In [ ]:
for i in range(len(inf)):
    d = inf.iloc[i]
    if ',' in d['ФИО']:
        print (
        d['ФИО'], '\n', d['Деревня'], '\n', d['Место рождения'], '\n', d['Год рождения'])

In [54]:
con.rollback()

Texts

In [59]:
df['Имя файла']

0        Архив ВШЭ-ФЭ-69-14-2017-1
1        Архив ВШЭ-ФЭ-69-14-2017-2
2        Архив ВШЭ-ФЭ-69-14-2017-3
3        Архив ВШЭ-ФЭ-69-14-2017-4
4        Архив ВШЭ-ФЭ-69-14-2017-5
5        Архив ВШЭ-ФЭ-69-14-2017-6
6        Архив ВШЭ-ФЭ-69-14-2017-7
7        Архив ВШЭ-ФЭ-69-14-2017-8
8        Архив ВШЭ-ФЭ-69-14-2017-9
9       Архив ВШЭ-ФЭ-69-14-2017-10
10      Архив ВШЭ-ФЭ-69-14-2017-11
11      Архив ВШЭ-ФЭ-69-14-2017-12
12      Архив ВШЭ-ФЭ-69-14-2017-13
13      Архив ВШЭ-ФЭ-69-14-2017-14
14      Архив ВШЭ-ФЭ-69-14-2017-15
15      Архив ВШЭ-ФЭ-69-14-2017-16
16      Архив ВШЭ-ФЭ-69-14-2017-17
17      Архив ВШЭ-ФЭ-69-14-2017-18
18      Архив ВШЭ-ФЭ-69-14-2017-19
19      Архив ВШЭ-ФЭ-69-14-2017-20
20      Архив ВШЭ-ФЭ-69-14-2017-21
21      Архив ВШЭ-ФЭ-69-14-2017-22
22      Архив ВШЭ-ФЭ-69-14-2017-23
23     Архив ВШЭ-ФЭ-69-14-2017-101
24     Архив ВШЭ-ФЭ-69-14-2017-102
25     Архив ВШЭ-ФЭ-69-14-2017-103
26     Архив ВШЭ-ФЭ-69-14-2017-104
27     Архив ВШЭ-ФЭ-69-14-2017-105
28     Архив ВШЭ-ФЭ-

Collectors

In [67]:
w = set()
for i in inf.values:
    d = i[0].split(', ')
    w.update(d)

In [ ]:
w

# Block 1 + 2 (finish)

Informators'gender

In [92]:
def g_name(name):
    global m
    res = m.analyze(name)
    fem, mal = 0, 0
    if len(res) == 6:
        res = res[2:]
    for i in res:
        if 'analysis' in i:
            #print (i['analysis'])
            if ',f,' in i['analysis'][0]['gr']:
                fem += 1
            elif ',m,' in i['analysis'][0]['gr']:
                mal += 1
    if fem > 0 and mal == 0:
        return 'f'
    elif fem == 0 and mal > 0:
        return 'm'
    else:
        return None

In [75]:
cur.execute('SELECT id, name FROM Informators')

In [76]:
inf = cur.fetchall()

In [98]:
for i in inf:
    cur.execute('UPDATE Informators  SET gender = ? WHERE id = ?', (g_name(i[1]), i[0]))
    #print (i[1], g_name(i[1]))

In [99]:
con.commit()

In [91]:
m.analyze('Соболькова Зоя Тимофеевна')

[{'analysis': [{'lex': 'собольков',
    'wt': 0.5946199918,
    'qual': 'bastard',
    'gr': 'S,famn,m,anim=(acc,sg|gen,sg)'}],
  'text': 'Соболькова'},
 {'text': ' '},
 {'analysis': [{'lex': 'зоя', 'wt': 1, 'gr': 'S,persn,f,anim=nom,sg'}],
  'text': 'Зоя'},
 {'text': ' '},
 {'analysis': [{'lex': 'тимофеевна', 'wt': 1, 'gr': 'S,patrn,f,anim=nom,sg'}],
  'text': 'Тимофеевна'},
 {'text': '\n'}]

Birthplace(block 1)